# Тема 18. Вентильные рекуррентные нейронные сети.
Рекуррентные сети, которые мы изучили на прошлом занятии, достаточно хороши. Позволяют делать модели сложных объектов, прогнозировать временные ряды и решать многие другие задачи. 

Но обычно, такие сети могут моделировать лишь краткосрочные зависимости на 10-100 тактов в прошлое. 

Но многие процессы имеют долгосрочные зависимости, например, в тексте: "Вася пошел погулять в лес"... затем 10 страниц описания того, что в этом лесу есть, а затем фраза "он увидел белочку". Кто такой "он" мы сможем понять только проанализировав текст на 10 страниц назад, а это довольно долгосрочная зависимость. Музыка, при частоте дискретизации 44000 Герц за 1 секунду наберется 44000 временных отсчетов, чтобы найти взаимосвязь между нотами, которые могут длиться до нескольких секунд потребуется обработать десятки тысяч отсчетов. 

С долгосрочными зависимостями обычные рекуррентные сети справляются очень плохо, почему?













# Проблема долгосрочных зависимостей
Вернемся к примеру одного рекуррентного нейрона, см. рисунок. 

![img](https://drive.google.com/uc?id=1TDrp7wnYk1gmkA-nbI29c5gnmQ4kPxHi)

Для простоты уберем функцию активации и смещение, получим формулу его работы: 

$ y(t)=W*y(t-1) $

Аналогично для любого другого времени:

$ y(t-1)=W*y(t-2) $

Подставив одно в другое получим:

$ y(t)=W*W*y(t-2) $ 

и далее по аналогии:

$ y(t)=W^n*y(t-n) $

То есть, чем дальше в прошлое, тем больше степень коэффициента влияние того отсчета **y(t-n)** на текущий **y(t)**.

Если |W|>1, то влияние далеких отсчетов будет возрастать очень быстро (например W=2, для тысячного отсчета в прошлое коэффициент будет $2^{1000} ), забивая влияние всех остальных отсчетов. Производная также будет быстро расти (а чему равна производная степенной функции?), это проблема **"взрывного градиента"**. Конечно, с ней можно бороться, например принудительно ограничить величину градиента, но все равно, далекие отсчеты будут влиять очень-очень сильно, а близкие гораздо слабее.

Но еще больше проблем возникает в случае, если |W| < 1.
Тогда влияние далеких отсчетов будет уменьшаться очень быстро (например W=0.5, для тысячного отсчета в прошлое коэффициент будет $2^{-1000} ), производная также будет быстро убывать и далекие отсчеты влияния практически не оказывают. Это проблема **"исчезающего градиента"**, решить ее очень трудно даже для простейшего рекуррентного нейрона, не говоря уж о более сложных многослойных рекуррентных сетях.





# Вентильные сети
Нужны другие подходы для рекуррентных сетей, в которых мы могли бы регулировать степень влияния разных прошлых отсчетов на текущий по нашему желанию.

А давайте так и сделаем, введем коэффициент влияния прошлых отсчетов на текущий (от 0 до 1), причем сделаем его обучаемым, чтобы в процессе обучения он подбирался нужным. Нужно сделать долгосрочные зависимости - ставь коэффициент близким к 1. Нужны краткосрочные - близким к 0.

Коэффициент это множитель, вычислительный блок с таким множителем назвали **"вентиль"** (gate).

Пусть у нашего нейрона будет некоторое *состояние* $C_t$, число, описывающее его текущее состояние в момент времени t. Мы будем регулировать это состояние.

Текущее состояние $C_t$ определяется предыдущим с некоторым множителем $ f_t $:

$ C_t=f_t * C_{t-1} $

Такой множитель, от 0 до 1, определяет какую часть состояния нужно "забыть", а соответствующий вентиль называют вентилем забывания (кружочек с крестиком на рисунке).

Но надо также добавлять новую информацию, полученную на текущем такте времени, поэтому введем еще один вентиль (вентиль добавления, это сумматор, кружочек с плюсиком на рисунке), который будет добавлять информацию и тогда текущее состояние определяется как 

$ C_t=f_t * C_{t-1} + С_{new} $



![img](https://drive.google.com/uc?id=14rgv8W36pjWMzqfXofhLcIqx94UKzUzH)



Если научиться регулировать коэффициент забывания и величину, добавляемую в состояние, то можно регулировать и состояние нейрона. Необязательно, но можно, также различать между собой состояние и выход нейрона. Для общности будем считать, что выход нейрона $h_t$ есть некоторая функция от состояния, например, гипертангенс с некоторым множителем:

$h_t=o_t*tanh(C_t)$


Коэффициент забывания $f_t$ должен быть в диапазоне от 0 до 1 и должен регулироваться в процессе обучения. А давайте в качестве такого коэффициента возьмем выход обычного нейрона с функцией активации "сигмоида" (сигма). Диапазон изменения будет нужный, а изменять выход можем изменением входов в нейрон и его весов. Такой коэффициент будет зависеть от текущего входа $x_t$, предыдущего выхода $h_{t-1}$ и весов этого нейрона:

$f_t=\sigma(W_{fh}*h_{t-1}+W_{fx}*x_t+b_f)$

![img](https://drive.google.com/uc?id=16LcYt4Kc_0ymnSnebM58CohnlAf6PLIa)

Мы получили основную часть **"вентильного нейрона"**. Теперь можем обучая веса нейрона в вентиле забывания подбирать коэффициент забывания под нужную длительность зависимостей.

Нейронные сети, состоящие из вентильных нейронов называют вентильными нейронными сетями. Сегодня их придумано десятки тысяч. Пожалуй самой известной среди них является **LSTM-сеть** (Long Short Term Memory, долгая краткосрочная память).

![img](https://drive.google.com/uc?id=1HrcawG2u8e5xRBcusSGZOHWKMneNs6Uf)

В этой сети в одном вентильном нейроне используется аж 4 обычных, которые определяют коэффициенты для вентилей забывания $f_t$, добавления $i_t, \tilde C_t$ и выхода  $o_t$. Детальное объяснение каждого вентиля смотри, например, [здесь](https://habr.com/ru/company/wunderfund/blog/331310/).

В другой реализации вентильного нейрона, GRU (Gated Recurrent Unit, вентильный рекуррентный блок), не стали различать состояние и выход, обошлись меньшим числом вентилей:

![img](https://drive.google.com/uc?id=1G_SkmXzMBNt4nUg0Q5y7U1NX8EBwxZDY)


## Обучение вентильных рекуррентных сетей
Если вы заметили, то все операции, которые выполняются в вентильном нейроне неважно какого вида - дифференцируемые. Поэтому процесс обучения такой сети ничем не отличается от обучения рекуррентной сети: разворачиваем во времени, применяем метод обратного распространения ошибки для расчета производной, изменяем веса. Отличие только в архитектуре нейрона и сети. Так в LSTM нейроне нужно обучать четыре набора весов и смещений, в GRU три. 

Нельзя сказать, что какая-то вентильная сеть лучше другой, это справедливо только для конкретной задачи, на которой вы можете их сравнить. Некоторые ученые вообще задумались, а зачем нам решать, давайте заставим компьютер перебирать всевозможные комбинации соединений вентилей и получим лучший вентильный нейрон для нашей задачи. Таким образом можно создать тысячи и десятки тысяч разных вентильных нейронов. В курсе мы ограничимся лишь LSTM сетями.




# Прогноз погоды
Что ж, давайте попробуем использовать вентильную LSTM сеть для предсказания погоды. Эта задача реализована в другой "тетрадке", [перейдите в нее](https://colab.research.google.com/drive/14Rtc8k844F6Yhc_PDApXyxlV6TsqkTZp). 

# Ссылки


https://habr.com/ru/company/wunderfund/blog/331310/
